# Neutral-atom Hardware

What you will learn in this section:
- What are the constraints introduced by a `Device` ? by a `Channel` ?
- What are the examples of `Device` in Pulser ?

## The `Device`: An object to store all the hardware constraints

As presented in the [introduction to programming a neutral-atom QPU](programming.md#programming-a-neutral-atom-qpu), the first step to [writing a Pulser program](programming.md#writing-a-pulser-program) is the selection of a `Device`. The `Device` object 
stores all the physical constraints a quantum program written with `Pulser` should verify, for it to be run on a neutral-atom QPU. The quantum programs, the `pulser.Sequence`, verify that each operation added to them are valid with respect to the constraints of the `Device`.  

### Where to find the `Devices` that can be used for your quantum program

:::{important}
The `Device` represents the physics of a neutral-atom QPU but is not the QPU itself. A QPU must be accessed via a `Backend`, which is presented [in this notebook](./apidoc/backend.rst).
:::

If you want to run a quantum program on a QPU, start by selecting the `Backend` associated with this QPU. From this `Backend` the `Device` associated with the QPU can be accessed via the `get_available_devices` method.

If you don't have access to a QPU or don't have particular QPU in mind, `Pulser` provides examples of typical physical devices in `pulser.devices`. Notably, `pulser.AnalogDevice` is an example of a QPU implementing an [Ising Hamiltonian](./programming.md#ising-hamiltonian). 

If you want don't want to be bothered by any or part of the physical constraints during the writing of your quantum program, you can use a `VirtualDevice`. An example of such a `VirtualDevice` is the `MockDevice` provided in the `pulser.devices`, which gives full liberty to write a quantum program. `VirtualDevice` is detailed in [an advanced tutorial](./tutorials/virtual_devices.nblink).

:::{note}
It is possible to change the set of constraints with which a `Sequence` was built, by using `Sequence.switch_device`. This is especially useful if you built your `Sequence` with an example of a `Device` like `AnalogDevice`, and now want to run it on a QPU, or if you want to run it on a different QPU than initially planned.
::: 

:::{figure} files/decision_diagram_device.png
:align: center
:alt: Decision Diagram to select a Device for the computation
:width: 600
:::


#### How to select your `Device`: Specifications of a `Device`


Now that you know where to look for `Devices`, you might want know how to choose the one matching your usecase. The device specifications are here to guide your choice:

In [ ]:
import pulser

print(pulser.AnalogDevice.specs)

The `Device` is going to constrain the next steps of your [quantum program](./programming.md#writing-a-pulser-program):

- "Register" and "Layout" parameters are going to constrain the creation of your `Register`, that is, the number of atoms you can use and how you can place them in place. [As a reminder](programming.md#create-the-register), this impacts the [interaction strength in the interaction Hamiltonian](programming.md#interaction-hamiltonian). The creation of a `Register` is presented here. If the `Device` requires a `Layout` (as in the example here), then you have to associate a `RegisterLayout` to the `Register`, which adds more constraints. Check [this tutorial](./tutorials/reg_layouts.nblink) to see how to do it.

- Among the "Device" parameters, the "Rydberg level" is going to complete the determination the [interaction strength in the interaction Hamiltonian](programming.md#interaction-hamiltonian). This level determines the "Ising interaction coefficient", which is the $C_6$ coefficient of the [Ising Hamiltonian](./programming.md#ising-hamiltonian). The quantity $\frac{C_6}{\hbar}$ is accessible via `pulser.AnalogDevice.interaction_coeff`.

- The "Channels" are going to determine what [`Channels` are available for the computation](programming.md#pick-the-channels). The type of the available channels are going to define the "supported bases" and "supported states" among the Device parameters. Knowing what states you want to use in your computation, you can first check that they are among the "supported states", then find the bases and their associated channel that enable to use these states using [the conventions page](conventions.md#bases).

- The "Maximum sequence duration" constrains the duration of the [`Pulses` you can add](programming.md#4-add-the-pulses), and therefore the Hamiltonian describing the system can at most be defined between 0 and this value.

- The "Maximum number of runs" limits the number of runs a quantum program can be executed on the QPU. See the section on `Backends` to read more about this.

Here, `pulser.AnalogDevice` only supports the $\left|r\right>$ and $\left|g\right>$ states, because it only supports one channel of type "Rydberg". It implements the [Ising Hamiltonian](programming.md#ising-hamiltonian), that can be defined between 0 and 4000ns max. The `Channel` can be declared using its name "rydberg_global", but this object contains a lot more constraints. Let's dive into them. 

:::{note}
Other parameters are being defined in the `Device`. You can see the effect of each them in the [API documentation](./apidoc/core.rst#Devices).
:::

## Channels

The channels available for selection are stored in the `channels` property of the `Device`. Let's have a look at the only channel available in `AnalogDevice`, the "rydberg_global" channel:

In [ ]:
import pulser

print(pulser.AnalogDevice.channels["rydberg_global"])

The channel is characterized by:

- Its type, `Rydberg`, that defines the [states](conventions.md#bases) that can be addressed by the [driving Hamiltonian](programming.md#driving-hamiltonian) if this channel is picked. The `Rydberg` channel defines $\left|b\right>=\left|r\right>$ and $\left|a\right>=\left|g\right>$. All the type of channels can be found [here](conventions.md#bases).
- Its addressing, `Global`, which means that any Pulse added to this channel will implement the same driving Hamiltonian on all the atoms.
- Other parameters, that are going to set constraints on the quantities of the `Pulses` that can be added to the channel:
    - the **duration** of the pulse is constrained by the minimum and maximum pulse duration, as well as the clock period (it has to be a multiple of the clock period).
    - the **amplitude** is limited by the maximum amplitude.
    - the **detuning** is limited by the maximum absolute detuning.

:::{note}
The modulation bandwidth and the EOM support are associated with more advanced features explained [in this tutorial](./tutorials/output_mod_eom.nblink). For a full list of the parameters of a `Channel` and the constraints they set, please check the [API documentation](./apidoc/core.rst#channels).
:::